In [3]:
import numpy as np
import pandas as pd

In [4]:
# TRSC = "流水数据/table_loss_transaction_trainingdata.txt"
TRSC = "流水数据/table_loss_transaction_testdata.txt"
# TP = "抛出点数据/table_throwpoint_training.txt"
TP = "抛出点数据/table_throwpoint_test.txt" 

M = "商户数据/table_datamining_merchant.txt"
# LBL = "特征值/lable.csv"
# TRSC = "特征值/trsc_cut.csv"
# EIGEN = "特征值/eigen_train.csv"
EIGEN = "特征值/eigen_test.csv"

In [8]:
'''trsc: 读入drop后的transaction数据 '''
# try:
#     trsc
# except NameError:
#     trsc_exists = False
#     trsc=pd.read_csv(TRSC,engine='python')
# else:
#     trsc_exists = True
# trsc
trsc=pd.read_csv(TRSC,engine='python')
trsc.drop(columns=['id','terminal_id','payer_uid','received_amount','paid_amount','bankcard_credit','wallet_alipay_finance',
                   'alipay_point','hongbao_wosai','hongbao_wosai_mch','discount_wosai','discount_wosai_mch','hongbao_channel_mch',
                   'hongbao_channel_mch_top_up','card_pre','card_balance','merchant_sn','is_liquidation_next_day','pay_way','sub_pay_way','ctime'],inplace=True)

## 特征6： 交易类型、订单状态（成功与否）
**交易状态statue**：

创建 0-----> 0

成功 2000---> 1

其他 2001---> 0

**交易类型type**：

付款 30 ----> 1

取消 10 ----> 0

退款 11 ----> 0

In [9]:
# trsc.drop(columns=['Unnamed: 0'],inplace=True)
trsc['status'].replace(2000,1,inplace=True)
trsc['status'].replace(2001,0,inplace=True)
trsc['type'].replace(30,1,inplace=True)
trsc['type'].replace(10,0,inplace=True)
trsc['type'].replace(11,0,inplace=True)

## 特征2：（支付通道中）是否优惠

price_reduced = bool(#original_amount - #effective_amount)

price_reduced = 1 : 有优惠

price_reduced = 0 : 无优惠

In [10]:
trsc['price_reduced']= trsc['original_amount']-trsc['effective_amount']

In [11]:
#确认没有负值
for i in trsc.price_reduced:
    if i<0:
        print("@")

In [12]:
trsc['if_price_reduced'] = trsc['price_reduced'].astype(bool).astype(int)

In [13]:
 sum(trsc.if_price_reduced)

865

## 特征3： 提供的支付方式个数

（4*1vector) 

#payw_offer = sum[bool(bankcard_debit), bool(wallet_weixin), bool(wallet_alipay), bool(alipay_huabei)]


In [14]:
payw_sum = trsc.groupby(['store_id']).mean().reset_index()
payw_sum['bankcard_debit'] = payw_sum['bankcard_debit'].astype(bool).astype(int)
payw_sum['wallet_alipay'] = payw_sum['wallet_alipay'].astype(bool).astype(int)
payw_sum['wallet_weixin'] = payw_sum['wallet_weixin'].astype(bool).astype(int)
payw_sum['alipay_huabei'] = payw_sum['alipay_huabei'].astype(bool).astype(int)
payw_sum['payw_offer']=payw_sum['bankcard_debit']+payw_sum['wallet_alipay']+payw_sum['wallet_weixin']+payw_sum['alipay_huabei']
payw_sum
# trsc = trsc.merge(payw_sum[['store_id','payw_offer']],on='store_id')

,store_id,type,status,effective_amount,original_amount,bankcard_debit,wallet_weixin,wallet_alipay,alipay_huabei,discount_channel,discount_channel_mch,discount_channel_mch_top_up,hongbao_channel,pt,price_reduced,if_price_reduced,payw_offer
0,000ca856-74d4-4cbd-88a2-a1a449a20a35,1.000000,0.990200,1419.488709,1419.488709,1,1,1,1,0.566255,0.000000,0.0,0.069876,2.018051e+07,0.000000,0.000000,4
1,00154e48-23cf-ce40-76b1-476932d7785f,1.000000,0.303030,610742.424242,610742.424242,0,1,1,1,2.000000,0.000000,0.0,0.000000,2.018052e+07,0.000000,0.000000,3
2,0025b90a-5307-430e-a52d-5031bdd9cb26,0.991736,6.413223,27667.245179,27667.245179,1,1,1,1,1.794766,0.000000,0.0,0.002755,2.018049e+07,0.000000,0.000000,4
3,0031d846-366f-4d1e-1cff-fe962c0f40fe,0.997376,4.742020,2967.881504,2967.885877,1,1,1,1,1.292304,0.000000,0.0,0.054657,2.018046e+07,0.004373,0.000219,4
4,0057373d-a954-4e4f-a13a-cf7f091fb1a5,1.000000,0.446154,152355.076923,152355.076923,0,0,1,1,0.000000,0.000000,0.0,0.000000,2.018049e+07,0.000000,0.000000,2
5,005be1f9-0f80-de6b-9bf8-2b12999746f9,1.000000,0.275862,150910.379310,150910.379310,0,0,1,1,0.000000,0.000000,0.0,0.000000,2.018050e+07,0.000000,0.000000,2
6,00750016-7600-475a-911a-34a10af96849,1.000000,28.424837,16202.287582,16202.287582,1,1,1,1,0.496732,0.000000,0.0,2.411765,2.018048e+07,0.000000,0.000000,4
7,0076c9f4-45cd-429a-8dc9-efb3b40a5e31,1.000000,0.879004,7922.199288,7922.199288,1,1,1,1,1.106762,0.000000,0.0,0.000000,2.018048e+07,0.000000,0.000000,4
8,0080219d-57e7-6769-9cca-1640f6d6145d,1.000000,0.714286,53862.857143,53862.857143,1,1,1,1,2.142857,0.000000,0.0,0.114286,2.018048e+07,0.000000,0.000000,4
9,008b842b-34d7-4810-adf3-82e7fc75d355,1.000000,0.950704,2380.457746,2380.457746,1,1,1,1,1.855634,0.000000,0.0,0.477113,2.018044e+07,0.000000,0.000000,4


In [15]:
trsc.drop(columns=['effective_amount','bankcard_debit','wallet_weixin','wallet_alipay','alipay_huabei'],inplace=True)

## 特征4：（商户提供的）活动

bool(discount_channel & discount_channel &...       )

In [16]:
payw_sum['discount']=payw_sum['discount_channel'].astype(bool).astype(int)+payw_sum['discount_channel_mch'].astype(bool).astype(int)+payw_sum['hongbao_channel'].astype(bool).astype(int)+payw_sum['discount_channel_mch_top_up'].astype(bool).astype(int)
trsc = trsc.merge(payw_sum[['store_id','discount']],on='store_id')

In [17]:
payw_sum['if_discount']=payw_sum['discount_channel'].astype(bool)+payw_sum['discount_channel_mch'].astype(bool)+payw_sum['hongbao_channel'].astype(bool)+payw_sum['discount_channel_mch_top_up'].astype(bool)
sum(payw_sum.if_discount)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


2270

In [18]:
trsc.drop(columns=['discount_channel','discount_channel_mch','discount_channel_mch_top_up','hongbao_channel'],inplace=True)

In [19]:
trsc

,type,status,store_id,merchant_id,original_amount,pt,price_reduced,if_price_reduced,discount
0,1,1,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,800,20180512,0,0,2
1,1,1,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,800,20180512,0,0,2
2,1,1,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,2200,20180512,0,0,2
3,1,1,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,800,20180512,0,0,2
4,1,1,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,800,20180408,0,0,2
5,1,1,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,800,20180512,0,0,2
6,1,1,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,1600,20180519,0,0,2
7,1,1,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,800,20180421,0,0,2
8,1,1,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,500,20180517,0,0,2
9,1,1,008b842b-34d7-4810-adf3-82e7fc75d355,cee01785-713d-4c89-9912-ec83fa749c43,1400,20180407,0,0,2


## 不看throwpoint后的交易数据：

In [20]:
# lable = pd.read_csv(LBL,engine='python')
# lable.drop(columns=['Unnamed: 0'],inplace=True)
trpt = pd.read_csv( TP, engine='python')
trsc = trsc.merge(trpt,on='store_id',suffixes=('','_trpt'))

In [21]:
trsc['pt'] = pd.to_datetime(trsc.pt, format='%Y%m%d')
trsc['pt_trpt'] = pd.to_datetime(trsc.pt_trpt, format='%Y%m%d')

In [22]:
import datetime

trsc['pt_delta'] =  trsc['pt_trpt']-trsc['pt']
trsc = trsc[(trsc[['pt_delta']]>datetime.timedelta(days=0)).all(axis=1)]
trsc.drop(columns=['pt_delta'],inplace=True)

## 特征5： 日均交易笔数、日均交易额、单店日均交易额在该行业占比

#operate_state 

#daily_amount

In [23]:
mch = pd.read_csv(M,engine='python',sep=' ')
# mch

In [24]:
trsc = trsc.merge(mch,on='merchant_id')
trsc['count'] = int(1)

In [25]:
store_amount_sum = trsc.groupby(['store_id']).sum().reset_index()
mch_amount_sum = trsc.groupby(['merchant_id']).sum().reset_index()
store_amount_sum['price_reduced_rate'] = store_amount_sum['if_price_reduced']/store_amount_sum['count']
store_amount_sum['reduce_range'] = store_amount_sum['price_reduced']/store_amount_sum['original_amount']
store_amount_sum['pay_rate'] = store_amount_sum['type']/store_amount_sum['count']
store_amount_sum['success_rate'] = store_amount_sum['status']/store_amount_sum['count']

In [26]:
trsc['days'] = trsc['pt_trpt']-datetime.datetime.strptime('2018-06-01', '%Y-%m-%d')
trsc['days'] = trsc['days'].dt.days
type(trsc.days[1])

numpy.int64

In [27]:
eigen = trsc[['store_id','merchant_id','discount','industry_level1','province','days']]
eigen.drop_duplicates(inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
eigen = eigen.merge(store_amount_sum[['store_id','original_amount','count','price_reduced_rate','reduce_range','pay_rate','success_rate']],on='store_id')
eigen = eigen.merge(mch_amount_sum[['merchant_id','original_amount']],on='merchant_id',suffixes=('','_mch'))

In [29]:
eigen['operate_state'] = eigen['original_amount']/eigen['original_amount_mch']
eigen['daily_amount'] = eigen['original_amount']/eigen['days']
eigen['daily_count'] = eigen['count']/trsc['days']

## 特征1：单笔均额

mean(original_amount)

In [30]:
eigen.drop(columns=['merchant_id','days','original_amount','count','original_amount_mch'],inplace=True)
eigen = eigen.merge(payw_sum[['store_id','original_amount','payw_offer','']],on='store_id',suffixes=('','_mean'))

KeyError: "[''] not in index"

In [ ]:
eigen.rename(columns={'original_amount':'original_amount_mean'})
eigen.to_csv(EIGEN)